In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
chotot_df = pd.read_csv("chotot.txt")

from copy import deepcopy
chotot_df.hang = chotot_df.hang.replace(["Vinfast"], "VinFast")
all_hang = deepcopy(chotot_df.hang.unique())
all_hang = np.append(all_hang,['Thaco','McLaren','Cuu Long','Dongfeng','Dongben','Lamborghini','Vinaxuki','TMT',"Hoa Mai","Fuso"])

all_dong_xe = chotot_df.dong_xe.unique()
# Giá
chotot_gia = []
import math
for value in chotot_df.gia:
    chotot_gia.append(int(value.replace(".","").replace("đ","0")))
chotot_df['gia'] = chotot_gia
chotot_df[chotot_df['gia'] == 0]

# Kiểu dáng
chotot_df.kieu_dang = chotot_df.kieu_dang.replace(["Van","Minivan (MPV)","Coupe (2 cửa)","Pick-up (bán tải)"],["Van/Minivan", "Van/Minivan","Coupe","Bán tải / Pickup"])

chotot_df.info()
chotot_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29432 entries, 0 to 29431
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   dong_xe      29231 non-null  object
 1   gia          29432 non-null  int64 
 2   hang         29233 non-null  object
 3   hop_so       29224 non-null  object
 4   kieu_dang    27660 non-null  object
 5   km_da_di     22408 non-null  object
 6   nam_sx       29233 non-null  object
 7   nhien_lieu   27693 non-null  object
 8   so_cho_ngoi  28014 non-null  object
 9   tinh_trang   29429 non-null  object
 10  xuat_xu      25515 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.5+ MB


,dong_xe,gia,hang,hop_so,kieu_dang,km_da_di,nam_sx,nhien_lieu,so_cho_ngoi,tinh_trang,xuat_xu
0,Hiace,218000000,Toyota,Số sàn,Van/Minivan,999,2006,Dầu,16,Đã sử dụng,Nhật Bản
1,Spark,199999999,Chevrolet,Tự động,Hatchback,95000,2014,Xăng,5,Đã sử dụng,Mỹ
2,E Class,265000000,Mercedes Benz,Tự động,Sedan,170000,2003,Xăng,5,Đã sử dụng,Việt Nam
3,CLA Class,795000000,Mercedes Benz,Tự động,Sedan,65000,2015,Xăng,5,Đã sử dụng,Đức
4,Corolla Altis,420000000,Toyota,Tự động,Sedan,98000,2014,Xăng,5,Đã sử dụng,Việt Nam


In [ ]:
bonbanh_df = pd.read_csv("bonbanh.txt")
bonbanh_df = bonbanh_df.drop(columns=['kieu_dang'])
bonbanh_df = bonbanh_df.rename(columns={"gia" : "hon_hop"})
bonbanh_df = bonbanh_df.rename(columns={"dong_xe" : "kieu_dang"})
bonbanh_df.head()

,kieu_dang,hon_hop,hop_so,km_da_di,mau_xe,nhien_lieu,so_cho_ngoi,so_cua,tinh_trang,xuat_xu
0,Sedan,Xe\tMercedes Benz S class\tS450L Luxury\t2019\...,Số tự động,"9,000 Km",Đen,Xăng\t3.0 L,5 chỗ,4 cửa,Xe đã dùng,Lắp ráp trong nước
1,Sedan,Xe\tMercedes Benz C class\tC200\t2017\t- 1 Tỷ ...,Số tự động,"20,000 Km",Đỏ,Xăng\t2.0 L,5 chỗ,4 cửa,Xe đã dùng,Lắp ráp trong nước
2,Crossover,Xe\tHyundai Kona\t2.0 AT\t2021\t- 579 Triệu,Số tự động,0 Km,Đỏ,Xăng\t2.0 L,5 chỗ,5 cửa,Xe mới,Lắp ráp trong nước
3,Crossover,Xe\tHyundai Tucson\t2.0 AT Đặc biệt\t2021\t- 8...,Số tự động,0 Km,Trắng,Xăng\t2.0 L,5 chỗ,5 cửa,Xe mới,Lắp ráp trong nước
4,Sedan,Xe\tPorsche Panamera\t3.0 V6\t2017\t- 5 Tỷ 750...,Số tự động,"32,000 Km",Đen,Xăng\t3.0 L,4 chỗ,5 cửa,Xe đã dùng,Nhập khẩu


In [ ]:
columns = ['dong_xe','gia','hang','hop_so','kieu_dang','km_da_di','nam_sx','nhien_lieu','so_cho_ngoi','tinh_trang','xuat_xu']


In [ ]:
# tách hon_hop thành giá, năm sản xuất, hãng
x = set()
gia = []
nam_sx = []
hang = []
dong_xe = []

def process_branch(sent):
    sent = sent.replace("Dong Ben", "Dongben")
    sent = sent.replace("Dong Feng", "Dongfeng")
    sent = sent.replace("DongFeng", "Dongfeng")
    sent = sent.replace("DONGFENG", "Dongfeng")

    sent = sent.replace("cửu long", "Cuu Long")
    sent = sent.replace("Cửu Long", "Cuu Long")
    sent = sent.replace("Cửulong", "Cuu Long")
    sent = sent.replace("CuuLong", "Cuu Long")
    return(sent)


for idx, value in enumerate(bonbanh_df['hon_hop']):
    nam, gia_tien = value.split("\t")[-2:]
    nam_sx.append(nam)

    gia_tien = gia_tien.split()[1:]
    gia_tri = []
    for idx, gt in enumerate(gia_tien):
        if gt == "Triệu":
            gia_tri.append(int(gia_tien[idx-1])*1e6)
        if gt == "Tỷ":
            gia_tri.append(int(gia_tien[idx-1])*1e9)
    gia_tri = int(sum(gia_tri))
    gia.append(gia_tri)

    hang_xe = process_branch(value.split("\t")[1:-2][0]).split()
    #print(hang_xe)
    bonbanh_hangxe = ""
    flag = False
    for i in hang_xe:
        bonbanh_hangxe += i
        if bonbanh_hangxe in all_hang:
            #print(bonbanh_hangxe)
            hang.append(bonbanh_hangxe)
            flag = True
            break
        else:
            bonbanh_hangxe += " "
    if flag == False:
        hang.append(bonbanh_hangxe)

    dong = hang_xe[len(bonbanh_hangxe.split()):]
    dong_xe_tmp = ""
    if len(dong) > 0:
        for i in dong:
            dong_xe_tmp += i
            if dong_xe_tmp in all_dong_xe:
                #print(bonbanh_hangxe)
                dong_xe.append(dong_xe_tmp)
                dong_xe_tmp = ""
                break
            else:
                bonbanh_hangxe += " "
        if len(dong_xe_tmp) > 1:
            dong_xe.append(dong_xe_tmp)
    else:
        dong_xe.append(float('NaN'))

# nhiên liệu
nhien_lieu = []
for idx, value in enumerate(bonbanh_df['nhien_lieu']):
    nhien_lieu.append(value.split()[0])

# so_cho_ngoi
so_cho_ngoi = []
for idx, value in enumerate(bonbanh_df['so_cho_ngoi']):
    so_cho_ngoi.append(value.split()[0])

# tình trang
bonbanh_df["tinh_trang"] = bonbanh_df['tinh_trang'].replace(['Xe đã dùng','Xe mới'],['Đã sử dụng',"Mới"])

# km đã đi
km_da_di = []
for idx, value in enumerate(bonbanh_df['km_da_di']):
    km_da_di.append(int(value.split()[0].replace(",","")))
    
bonbanh_df.hop_so = bonbanh_df["hop_so"].replace(["Số tự động"], "Tự động")
bonbanh_df["km_da_di"] = km_da_di
bonbanh_df["so_cho_ngoi"] = so_cho_ngoi
bonbanh_df["nhien_lieu"] = nhien_lieu
bonbanh_df["nam_sx"] = nam_sx
bonbanh_df["gia"] = gia
bonbanh_df["dong_xe"] = dong_xe
bonbanh_df = bonbanh_df.drop(columns=['hon_hop'])
bonbanh_df["hang"] = hang
bonbanh_df.kieu_dang = bonbanh_df.kieu_dang.replace(["Crossover","SUV"],["SUV / Cross over", "SUV / Cross over"])
bonbanh_df

,kieu_dang,hop_so,km_da_di,mau_xe,nhien_lieu,so_cho_ngoi,so_cua,tinh_trang,xuat_xu,nam_sx,gia,dong_xe,hang
0,Sedan,Tự động,9000,Đen,Xăng,5,4 cửa,Đã sử dụng,Lắp ráp trong nước,2019,4480000000,Sclass,Mercedes Benz
1,Sedan,Tự động,20000,Đỏ,Xăng,5,4 cửa,Đã sử dụng,Lắp ráp trong nước,2017,1155000000,Cclass,Mercedes Benz
2,Crossover,Tự động,0,Đỏ,Xăng,5,5 cửa,Mới,Lắp ráp trong nước,2021,579000000,Kona,Hyundai
3,Crossover,Tự động,0,Trắng,Xăng,5,5 cửa,Mới,Lắp ráp trong nước,2021,825000000,Tucson,Hyundai
4,Sedan,Tự động,32000,Đen,Xăng,4,5 cửa,Đã sử dụng,Nhập khẩu,2017,5750000000,Panamera,Porsche
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35969,SUV,Tự động,0,Đỏ,Xăng,5,5 cửa,Đã sử dụng,Lắp ráp trong nước,2020,870000000,CX5,Mazda
35970,Sedan,Số tay,0,Cát,Xăng,5,4 cửa,Đã sử dụng,Lắp ráp trong nước,2001,185000000,Camry,Toyota
35971,Sedan,Tự động,8000,Trắng,Xăng,5,4 cửa,Đã sử dụng,Lắp ráp trong nước,2020,529000000,Accent,Hyundai
35972,SUV,Tự động,120000,Ghi,Xăng,5,5 cửa,Đã sử dụng,Nhập khẩu,2005,495000000,RX,Lexus


In [ ]:
chotot_df.kieu_dang.unique()

array(['Van/Minivan', 'Hatchback', 'Sedan', 'SUV / Cross over',
       'Kiểu dáng khác', nan, 'Bán tải / Pickup', 'Coupe', 'Mui trần'],
      dtype=object)

In [ ]:
bonbanh_df.kieu_dang.unique()

array(['Sedan', 'SUV / Cross over', 'Bán tải / Pickup', 'Truck',
       'Van/Minivan', 'Hatchback', 'Coupe', 'Wagon',
       'Convertible/Cabriolet'], dtype=object)